In [1]:
import numpy as np
import matplotlib.pyplot as plt
import desc.io
from desc.grid import LinearGrid, ConcentricGrid
from desc.objectives import (
    ObjectiveFunction,
    FixBoundaryR,
    FixBoundaryZ,
    FixPressure,
    FixCurrent,
    FixIota,
    FixPsi,
    AspectRatio,
    ForceBalance,
    QuasisymmetryTwoTerm,
    RotationalTransform
)
from desc.optimize import Optimizer
from desc.plotting import (
    plot_grid,
    plot_boozer_modes,
    plot_boozer_surface,
    plot_qs_error,
    plot_boundaries,
    plot_boundary
)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


DESC version 0+untagged.9458.g5c309d8.dirty,using JAX backend, jax version=0.4.28, jaxlib version=0.4.28, dtype=float64
Using device: CPU, with 5.43 GB available memory


In [2]:
eq_fam = desc.io.load("inputs/nfp2_QA.h5")
eq_init = eq_fam[-1]
eq_qs = eq_init.copy()

'''idx_Rcc = eq_init.surface.R_basis.get_idx(M=1, N=2)
idx_Rss = eq_init.surface.R_basis.get_idx(M=-1, N=-2)

idx_Zsc = eq_init.surface.Z_basis.get_idx(M=-1, N=2)
idx_Zcs = eq_init.surface.Z_basis.get_idx(M=1, N=-2)

R_modes = np.delete(eq_init.surface.R_basis.modes, [idx_Rcc, idx_Rss], axis=0)
Z_modes = np.delete(eq_init.surface.Z_basis.modes, [idx_Zsc, idx_Zcs], axis=0)'''


'idx_Rcc = eq_init.surface.R_basis.get_idx(M=1, N=2)\nidx_Rss = eq_init.surface.R_basis.get_idx(M=-1, N=-2)\n\nidx_Zsc = eq_init.surface.Z_basis.get_idx(M=-1, N=2)\nidx_Zcs = eq_init.surface.Z_basis.get_idx(M=1, N=-2)\n\nR_modes = np.delete(eq_init.surface.R_basis.modes, [idx_Rcc, idx_Rss], axis=0)\nZ_modes = np.delete(eq_init.surface.Z_basis.modes, [idx_Zsc, idx_Zcs], axis=0)'

In [3]:
optimizer = Optimizer("pdfo")

grid_vol = ConcentricGrid(
    L=eq_init.L_grid,
    M=eq_init.M_grid,
    N=eq_init.N_grid,
    NFP=eq_init.NFP,
    sym=eq_init.sym
)

In [4]:
for maxMode in range(1,2):
    objective_f = ObjectiveFunction((
        QuasisymmetryTwoTerm(eq=eq_qs, grid=grid_vol, helicity=(1, 0), weight=1),
        AspectRatio(eq=eq_qs, target=6, weight=100),
        RotationalTransform(eq=eq_qs, target=0.42, weight=100))
    )
    
    R_modes = np.vstack(([0,0,0], eq_qs.surface.R_basis.modes[np.max(np.abs(eq_qs.surface.R_basis.modes), 1) > maxMode, :]))
    Z_modes = eq_qs.surface.Z_basis.modes[np.max(np.abs(eq_qs.surface.Z_basis.modes), 1) > maxMode, :]
    
    constraints = (
        ForceBalance(eq=eq_qs),
        FixBoundaryR(eq=eq_qs, modes=R_modes),
        FixBoundaryZ(eq=eq_qs, modes=Z_modes),
        FixPressure(eq=eq_qs),
        FixCurrent(eq=eq_qs),
        FixPsi(eq=eq_qs)
    )
    
    eq_qs, result = eq_qs.optimize(
        objective=objective_f,
        constraints=constraints,
        optimizer=optimizer,
        maxiter=20,
        copy=True,
        verbose=2
    )
    
    plot_boundary(eq_qs, figsize=(8,8))
    plot_boozer_surface(eq_qs, figsize=(8,8))

Building objective: QS two-term
Precomputing transforms
Timer: Precomputing transforms = 1.11 sec
Building objective: aspect ratio
Precomputing transforms
Timer: Precomputing transforms = 33.3 ms
Building objective: rotational transform
Precomputing transforms
Timer: Precomputing transforms = 178 ms
Timer: Objective build = 4.80 sec
Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed pressure
Building objective: fixed current
Building objective: fixed Psi
Building objective: self_consistency R
Building objective: self_consistency Z
Building objective: lambda gauge
Building objective: axis R self consistency
Building objective: axis Z self consistency
Timer: Objective build = 1.91 sec
Building objective: force
Precomputing transforms
Timer: Precomputing transforms = 304 ms
Timer: Objective build = 490 ms
Timer: Linear constraint projection build = 3.13 sec
Timer: Linear constraint projection build = 35.6 ms
Number of parameters: 155
Number of objectives: 535


/home/ryc/.local/lib/python3.10/site-packages/pdfo/_common.py:1637: Warning: pdfo: invalid maxfev; it should be a positive integer at least n+2; it is set to n+2.
  warnings.warn(warn_message, Warning)


ValueError: assignment destination is read-only

In [ ]:
eq_qs.current